In [28]:
import geopandas as gpd
import pandas as pd
import requests
from lxml import html

In [19]:
postcode = list(gpd.read_file("../data/ABS/post_shapefile/POA_2021_AUST_GDA2020.shp")["POA_CODE21"])
consumer = list(pd.read_csv("../data/tables/tbl_consumer.csv", delimiter="|")["postcode"].unique())
consumer = [str(i).rjust(4, '0') for i in consumer]

In [20]:
to_do = [i for i in consumer if i not in postcode]
to_do

['6935',
 '1109',
 '6849',
 '1114',
 '1825',
 '1129',
 '6963',
 '5071',
 '1860',
 '8070',
 '1150',
 '9005',
 '6723',
 '8009',
 '6981',
 '6951',
 '1176',
 '1134',
 '0851',
 '3619',
 '1427',
 '6918',
 '6435',
 '6903',
 '1046',
 '6332',
 '1162',
 '4219',
 '6846',
 '1314',
 '1350',
 '1225',
 '0907',
 '1485',
 '1236',
 '1210',
 '1186',
 '4471',
 '8011',
 '1169',
 '1426',
 '6761',
 '6947',
 '1213',
 '1235',
 '0871',
 '1800',
 '2442',
 '1107',
 '1202',
 '4271',
 '6964',
 '1293',
 '1298',
 '1140',
 '1296',
 '1851',
 '6997',
 '6842',
 '5810',
 '1147',
 '1143',
 '1124',
 '6926',
 '6940',
 '1229',
 '8001',
 '6953',
 '1360',
 '6943',
 '3176',
 '6827',
 '2348',
 '1105',
 '1218',
 '2129',
 '1138',
 '0906',
 '4803',
 '1159',
 '6938',
 '2616',
 '0813',
 '1181',
 '8120',
 '6231',
 '3841',
 '1117',
 '2901',
 '2052',
 '6969',
 '1039',
 '8111',
 '1034',
 '1171',
 '1585',
 '3001',
 '1023',
 '6956',
 '6945',
 '1149',
 '3853',
 '6911',
 '8002',
 '1036',
 '8010',
 '4857',
 '6901',
 '6920',
 '1026',
 '6955',
 

In [123]:
def get_suburb(pobox: str):
    r = requests.get(f"https://auspost.com.au/postcode/{pobox}")
    tree = html.fromstring(r.content)
    suburb = tree.xpath('//td[@class="second"]//a[@class="result_1"]/text()')
    return suburb[0].replace(' DC,', ',')

def get_post(pobox: str):
    cur_sub = get_suburb(pobox)
    r = requests.get(f"https://auspost.com.au/postcode/{cur_sub.split(', ')[0].lower()}")
    tree = html.fromstring(r.content)
    postcode = tree.xpath('//td[@class="first"]//a/text()')
    suburb = tree.xpath('//td[@class="second"]//a/text()')
    type = tree.xpath('//td[@class="third"]//a/text()')
    cur = []
    for i in range(len(postcode)):
        if suburb[i] == cur_sub and postcode[i] != pobox and type[i] != "Post Office Boxes":
            cur += [postcode[i]]
    return cur

out = [['pobox', 'postcode']]
for i in to_do:
    try:
        out += [[i] + get_post(i)]
    except:
        # large volume receivers have been pretty much discontinued so these will be treated as outliers and skipped
        pass

In [124]:
# check that there is one to one mapping
for i in out:
    if len(i) > 2:
        print(i)

for i in out:
    if len(i) == 1:
        print(i)

['6849']
['8009']
['6332']
['4219']
['1225']
['8011']
['2442']
['4271']
['3176']
['2348']
['2129']
['4803']
['3841']
['2901']
['8010']
['2891']
['6850']
['6831']
['1230']
['2310']
['3010']
['4345']
['6905']
['7151']
['6966']
['1215']
['1515']
['2708']
['3050']
['1481']
['2252']
['1340']
['4222']
['8003']
['6731']
['4813']
['4230']
['0815']
['6904']
['1220']
['0200']
['2002']
['3694']
['4072']
['2012']
['2890']
['2608']
['2004']
['6960']
['6433']
['4002']
['3800']
['8007']
['2006']
['4003']
['0801']
['8004']
['4029']
['6733']
['8006']
['1466']
['4001']
['2309']
['2610']
['6531']
['9999']
['6711']
['1635']


In [125]:
def get_primary(post: str):
    r = requests.get(f"https://postal-codes.cybo.com/search/?q={post}")
    tree = html.fromstring(r.content)
    print(r.content)

get_primary("2901")

b'<!DOCTYPE html>\n<html lang="en-US">\n<head>\n    <title>Just a moment...</title>\n    <meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />\n    <meta http-equiv="X-UA-Compatible" content="IE=Edge" />\n    <meta name="robots" content="noindex,nofollow" />\n    <meta name="viewport" content="width=device-width,initial-scale=1" />\n    <link href="/cdn-cgi/styles/challenges.css" rel="stylesheet" />\n    \n\n</head>\n<body class="no-js">\n    <div class="main-wrapper" role="main">\n    <div class="main-content">\n        <h1 class="zone-name-title h1">\n            <img class="heading-favicon" src="/favicon.ico"\n                 onerror="this.onerror=null;this.parentNode.removeChild(this)" />\n            postal-codes.cybo.com\n        </h1>\n        <h2 class="h2" id="challenge-running">\n            Checking if the site connection is secure\n        </h2>\n        <noscript>\n            <div id="challenge-error-title">\n                <div class="h2">\n            